In [1]:
import boto3
import pandas as pd

# Nome do bucket e caminho do arquivo no S3
bucket_name = "recomendacao-produtos-aws"
file_key = "matriz_cliente_produto.csv"

# Criar um cliente S3
s3 = boto3.client("s3")

# Fazer o download do arquivo do S3 para o notebook
s3.download_file(bucket_name, file_key, "matriz_cliente_produto.csv")

# Carregar os dados
df = pd.read_csv("matriz_cliente_produto.csv")

# Mostrar as primeiras linhas
df.head()


,customer_unique_id,04c4a4b9c924494fcf82e0fba966f955,06edb72f1e0c64b14c5b79353f7abea3,0bcc3eeca39e1064258aa1e932269894,154e7e31ebfa092203795c972e5804a6,2b4609f8948be18874494203496bc318,368c6c730842d78016ad823897a372db,36f60d45225e60c7da4558b070ce4b60,386486367c1f9d4f587a8864ccb6902b,389d119b48cf3043d311335e499d9c6b,...,d34c07a2d817ac73f4caf8c574215fed,d493f5a208254fe85b3ab55d898461a1,d5991653e037ccb7af6ed7d94246b249,dbb67791e405873b259e4656bf971246,e0d64dcfaa3b6db5c54ca298ae101d05,e53e557d5a159f5aa2c5e995dfdf244b,e7cc48a9daff5436f63d3aad9426f28b,ec2d43cc59763ec91694573b31f1c29a,f1c7f353075ce59d8a6f3cf58f419c9c,f3720bc68555b1bff49b9ffd41b017ac
0,0178b244a5c281fb2ade54038dd4b161,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,02b20b7c813efede140142ac610e36dc,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,02fbe4db886eb98774c8ba300d39fa79,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,048dbdf8f9bbd1be87b25868d1408311,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,04d3c2795b83634312bf3f86f41f7901,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
!pip install scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Converter a matriz para o formato necessário (lista de interações)
interactions = df.melt(id_vars=["customer_unique_id"], var_name="product_id", value_name="purchase_count")

# Remover entradas sem compras (zeros)
interactions = interactions[interactions["purchase_count"] > 0]

# Definir os limites dos valores
reader = Reader(rating_scale=(1, interactions["purchase_count"].max()))

# Criar o dataset no formato do Surprise
data = Dataset.load_from_df(interactions[["customer_unique_id", "product_id", "purchase_count"]], reader)

# Dividir em treino e teste
trainset, testset = train_test_split(data, test_size=0.2)

# Criar e treinar o modelo SVD
model = SVD()
model.fit(trainset)

# Fazer previsões no conjunto de teste
predictions = model.test(testset)

# Avaliar o modelo (RMSE)
rmse = accuracy.rmse(predictions)
print(f"RMSE do modelo: {rmse:.4f}")


RMSE: 1.3185
RMSE do modelo: 1.3185


In [3]:
from surprise.model_selection import GridSearchCV

# Definir a grade de hiperparâmetros para otimização
param_grid = {
    "n_factors": [50, 100, 150],  # Número de dimensões latentes
    "reg_all": [0.02, 0.05, 0.1],  # Regularização
    "lr_all": [0.002, 0.005, 0.01]  # Taxa de aprendizado
}

# Criar o GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

# Executar a busca de hiperparâmetros
gs.fit(data)

# Melhor combinação encontrada
best_params = gs.best_params["rmse"]
print("Melhores hiperparâmetros:", best_params)

# Treinar o modelo com os melhores hiperparâmetros
best_model = SVD(n_factors=best_params["n_factors"], reg_all=best_params["reg_all"], lr_all=best_params["lr_all"])
best_model.fit(trainset)

# Avaliar o modelo otimizado
best_predictions = best_model.test(testset)
best_rmse = accuracy.rmse(best_predictions)
print(f"RMSE do modelo otimizado: {best_rmse:.4f}")


Melhores hiperparâmetros: {'n_factors': 100, 'reg_all': 0.02, 'lr_all': 0.01}
RMSE: 1.3201
RMSE do modelo otimizado: 1.3201


In [4]:
# Função para recomendar produtos para um cliente específico
def recomendar_produtos(customer_id, modelo, df, n=5):
    # Obter a lista de todos os produtos
    produtos_disponiveis = df.columns[1:]  # Ignorando a coluna de clientes
    
    # Fazer previsões para todos os produtos
    previsoes = [modelo.predict(customer_id, produto) for produto in produtos_disponiveis]
    
    # Ordenar pelos produtos com maior previsão de interesse
    previsoes.sort(key=lambda x: x.est, reverse=True)
    
    # Retornar os N produtos recomendados
    recomendados = [pred.iid for pred in previsoes[:n]]
    
    return recomendados

# Testar recomendação para um cliente
cliente_exemplo = df["customer_unique_id"].iloc[0]  # Pegar um cliente aleatório
recomendacoes = recomendar_produtos(cliente_exemplo, best_model, df)

print(f"Produtos recomendados para o cliente {cliente_exemplo}:")
print(recomendacoes)


Produtos recomendados para o cliente 0178b244a5c281fb2ade54038dd4b161:
['e7cc48a9daff5436f63d3aad9426f28b', '89b190a046022486c635022524a974a8', '3fdb534dccf5bc9ab0406944b913787d', 'd493f5a208254fe85b3ab55d898461a1', 'd5991653e037ccb7af6ed7d94246b249']


In [5]:
# Configuração do S3
bucket_name = "recomendacao-produtos-aws"
file_key = "olist_products_dataset.csv"  # Caminho correto do arquivo no S3

# Baixar o dataset de produtos do S3
s3.download_file(bucket_name, file_key, "olist_products_dataset.csv")

# Carregar os dados do dataset de produtos
products_df = pd.read_csv("olist_products_dataset.csv")

# Mostrar as primeiras linhas para conferir as colunas disponíveis
products_df.head()


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [6]:
# Filtrar apenas os produtos recomendados
produtos_recomendados = products_df[products_df["product_id"].isin(recomendacoes)]

# Exibir os detalhes dos produtos recomendados (ID e categoria)
produtos_recomendados[["product_id", "product_category_name"]]


,product_id,product_category_name
10963,d5991653e037ccb7af6ed7d94246b249,informatica_acessorios
15672,e7cc48a9daff5436f63d3aad9426f28b,telefonia
20835,89b190a046022486c635022524a974a8,moveis_decoracao
31437,3fdb534dccf5bc9ab0406944b913787d,fraldas_higiene
31628,d493f5a208254fe85b3ab55d898461a1,alimentos


In [7]:
import pickle

# Caminho local para salvar o modelo
modelo_local = "modelo_svd.pkl"

# Salvar o modelo treinado
with open(modelo_local, "wb") as f:
    pickle.dump(best_model, f)

print(f"Modelo salvo em: {modelo_local}")


Modelo salvo em: modelo_svd.pkl


In [8]:
# Caminho do modelo no S3
modelo_s3_path = "modelo_svd.pkl"  # Nome do arquivo no S3

# Fazer upload para o S3
s3.upload_file(modelo_local, bucket_name, modelo_s3_path)

print(f"Modelo carregado para S3: s3://{bucket_name}/{modelo_s3_path}")


Modelo carregado para S3: s3://recomendacao-produtos-aws/modelo_svd.pkl


In [10]:
# Caminho local do arquivo
arquivo_local = "predictor.py"

# Caminho no S3
arquivo_s3_path = "predictor.py"

# Fazer upload para o S3
s3.upload_file(arquivo_local, bucket_name, arquivo_s3_path)

print(f"Arquivo predictor.py carregado para S3: s3://{bucket_name}/{arquivo_s3_path}")


Arquivo predictor.py carregado para S3: s3://recomendacao-produtos-aws/predictor.py


In [11]:
import tarfile

# Criar um arquivo .tar.gz contendo o modelo
tar_model_path = "modelo_svd.tar.gz"

with tarfile.open(tar_model_path, "w:gz") as tar:
    tar.add("modelo_svd.pkl")

print(f"Modelo compactado: {tar_model_path}")


Modelo compactado: modelo_svd.tar.gz


In [12]:
# Caminho do novo arquivo no S3
modelo_s3_tar_path = "modelo_svd.tar.gz"

# Fazer upload para o S3
s3.upload_file(tar_model_path, bucket_name, modelo_s3_tar_path)

print(f"Modelo compactado carregado para S3: s3://{bucket_name}/{modelo_s3_tar_path}")


Modelo compactado carregado para S3: s3://recomendacao-produtos-aws/modelo_svd.tar.gz


In [13]:
import pickle

# Carregar o modelo salvo para testar
modelo_path = "modelo_svd.pkl"

try:
    with open(modelo_path, "rb") as f:
        modelo = pickle.load(f)
    print("✅ O modelo foi carregado com sucesso!")
except Exception as e:
    print("❌ Erro ao carregar o modelo:", e)


✅ O modelo foi carregado com sucesso!


In [14]:
import tarfile

# Listar os arquivos dentro do .tar.gz
tar_path = "modelo_svd.tar.gz"

with tarfile.open(tar_path, "r:gz") as tar:
    print("📂 Arquivos dentro do .tar.gz:", tar.getnames())


📂 Arquivos dentro do .tar.gz: ['modelo_svd.pkl']


In [15]:
import pickle

# Carregar o modelo salvo para testar
modelo_path = "modelo_svd.pkl"

try:
    with open(modelo_path, "rb") as f:
        modelo = pickle.load(f)
    print("✅ O modelo foi carregado com sucesso!")
except Exception as e:
    print("❌ Erro ao carregar o modelo:", e)


✅ O modelo foi carregado com sucesso!
